# LightGBM model with Airline dataset. Experiment 01

LightGBM regressor with logloss + Airline dataset

In [1]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score, mean_squared_error, mean_absolute_error, r2_score)
from scipy.stats import gmean

from libs.timer import Timer
from libs.loaders import load_fraud, load_iot, load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.conversion import _get_nominal_integer_dict, _convert_to_integer
print("System version: {}".format(sys.version))

System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


### Load data

In [2]:
%%time
df_plane = load_airline()
print(df_plane.shape)


(115069017, 14)
CPU times: user 1min 49s, sys: 15.8 s, total: 2min 5s
Wall time: 4min 45s


In [3]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [4]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


CPU times: user 2min 2s, sys: 5.39 s, total: 2min 8s
Wall time: 2min 6s


In [5]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [6]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 1min 8s, sys: 7.19 s, total: 1min 15s
Wall time: 1min 14s


In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [8]:
%%time
filter_func = lambda x: 1 if x > 0 else 0
df_plane_numeric['ArrDelayBinary'] = df_plane_numeric['ArrDelay'].map(filter_func)

CPU times: user 45.7 s, sys: 2.78 s, total: 48.5 s
Wall time: 47.9 s


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [10]:
%%time
#gets a random 80% of the entire set
X_train = df_plane_numeric.sample(frac=0.8, random_state=1)
print(X_train.shape)
#gets the left out portion of the dataset
X_test = df_plane_numeric.loc[~df_plane_numeric.index.isin(X_train.index)]
print(X_test.shape)

(92055214, 15)
(23013803, 15)
CPU times: user 1min 17s, sys: 15.1 s, total: 1min 32s
Wall time: 1min 31s


### Training

In [11]:
# Variables
KFOLD_SPLITS = 5

In [12]:
def loglikelood(y_true, y_pred):
    eval_result = log_loss(y_true, y_pred)
    eval_name = 'log_loss'
    is_bigger_better = False
    return eval_name, eval_result, is_bigger_better

In [13]:
def train_lightgbm(df, verbose=True):
    if verbose: print("Training data of size {}".format(df.shape))
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
    y = df['ArrDelayBinary'].as_matrix()
    
    n_splits = 5
    skf = KFold(n_splits=n_splits, random_state=77, shuffle=True)
    clfs = []
    
    if verbose: print("Computing boosted tree using {} kfold cross validation".format(n_splits))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index]

        clf = LGBMRegressor(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            nthread=20,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=77)
        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric=loglikelood)
        clfs.append(clf)
        
    return clfs

In [ ]:
#X_train_small = X_train.sample(frac=0.001, random_state=1)
#print(X_train_small.shape)

In [ ]:
%%time
clfs = train_lightgbm(X_train)

Training data of size (92055214, 15)
Computing boosted tree using 5 kfold cross validation
[1]	valid_0's log_loss: 0.687524
[2]	valid_0's log_loss: 0.672164
[3]	valid_0's log_loss: 0.658358
[4]	valid_0's log_loss: 0.646863
[5]	valid_0's log_loss: 0.636871
[6]	valid_0's log_loss: 0.628261
[7]	valid_0's log_loss: 0.620758
[8]	valid_0's log_loss: 0.618818
[9]	valid_0's log_loss: 0.611606
[10]	valid_0's log_loss: 0.605749
[11]	valid_0's log_loss: 0.600623
[12]	valid_0's log_loss: 0.598674
[13]	valid_0's log_loss: 0.596972
[14]	valid_0's log_loss: 0.592017
[15]	valid_0's log_loss: 0.590809
[16]	valid_0's log_loss: 0.589155
[17]	valid_0's log_loss: 0.584711
[18]	valid_0's log_loss: 0.58348
[19]	valid_0's log_loss: 0.582277
[20]	valid_0's log_loss: 0.580969
[21]	valid_0's log_loss: 0.579708
[22]	valid_0's log_loss: 0.578639
[23]	valid_0's log_loss: 0.575031
[24]	valid_0's log_loss: 0.571649
[25]	valid_0's log_loss: 0.568565
[26]	valid_0's log_loss: 0.566287
[27]	valid_0's log_loss: 0.565393
[

### Evaluation


In [26]:
def compute_prediction(clfs, df, verbose=True):    
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()            
    preds = []
    with Timer() as t:
        for clf in clfs:
            preds.append(np.clip(clf.predict(x), 0.0001, 0.9999))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval)        
    df['prediction'] = pred
    return df

In [ ]:
#X_test_small = X_test.sample(frac=0.001, random_state=1)
#print(X_test_small.shape)

In [ ]:
%%time
X_test_pred = compute_prediction(clfs, X_test)
X_test_pred.head(20)

Prediction took 21059.016 sec.

CPU times: user 3h 19min 12s, sys: 2h 31min 47s, total: 5h 51min
Wall time: 18min 9s


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Metrics

In [27]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    m_conf = confusion_matrix(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1, 'Confusion Matrix':m_conf}
    return report

In [28]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    m_logloss = log_loss(y_true, y_prob)
    report = {'AUC':m_auc, 'Log loss':m_logloss}
    return report

In [29]:
THRES = 0.5
threshold_func = lambda x: 0 if x<=THRES else 1


In [30]:
y_true = X_test_pred['ArrDelayBinary'].as_matrix()
y_prob = X_test_pred['prediction'].as_matrix()
y_pred = X_test_pred['prediction'].map(threshold_func).as_matrix()

In [31]:
report1 = classification_metrics_binary(y_true, y_pred)
report1

{'Accuracy': 0.75133740390495218,
 'Confusion Matrix': array([[10111709,  1981590],
        [ 3741082,  7179422]]),
 'F1': 0.71502788932867412,
 'Precision': 0.78369311163439148,
 'Recall': 0.65742588437310223}

In [32]:
report2 = classification_metrics_binary_prob(y_true, y_prob)
report2

{'AUC': 0.82696509141091779, 'Log loss': 0.510765718586633}